In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()


                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder local. If you are in a cluster initialize Optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


In [5]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df =op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

Downloaded 967 bytes
Creating DataFrame for foo.csv. Please wait...
Loading file using SparkSession
Successfully created DataFrame for 'foo.csv'


### Quantile Statistics

In [6]:
print(df.cols.min("billingId"))
print(df.cols.percentile(['billingId', 'price'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols.max("billingId"))
print(df.cols.median(["billingId","price"]))
print(df.cols.range(["billingId","price"]))
print(df.cols.std(["billingId","price"]))

min function took 1503.947 ms
111
drop_na function took 12.998 ms
cast function took 15.992 ms
drop_na function took 5.984 ms
cast function took 8.018 ms
percentile function took 278.254 ms
{'billingId': {0.05: 111.0, 0.25: 323.0, 0.5: 553.0, 0.75: 812.0, 0.95: 992.0}, 'price': {0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}}
max function took 1157.119 ms
992
drop_na function took 5.925 ms
cast function took 7.979 ms
drop_na function took 4.987 ms
cast function took 7.925 ms
median function took 176.528 ms
{'billingId': 553.0, 'price': 8.0}
max function took 1124.332 ms
min function took 1115.912 ms
max function took 1115.301 ms
min function took 1104.330 ms
range function took 4466.820 ms
{'billingId': {'min': 111, 'max': 992}, 'price': {'min': 1, 'max': 10}}
std function took 1195.261 ms
{'stddev_billingId': 280.1973510859008, 'stddev_price': 2.9528457876452054}


In [7]:
print(df.cols.min("*"))

min function took 1235.670 ms
{'min_id': 1, 'min_firstName': '(((   Heinrich )))))', 'min_lastName': '             GALiLEI', 'min_billingId': 111, 'min_product': '110790', 'min_price': 1, 'min_birth': '1899/01/01', 'min_dummyCol': '#'}


### Descriptive Statistics

In [8]:
print(df.cols.kurt("billingId"))
print(df.cols.mean("billingId"))
print(df.cols.skewness("billingId"))
print(df.cols.sum("billingId"))
print(df.cols.variance("billingId"))
print(df.cols.mad("billingId"))

kurt function took 1184.268 ms
-1.0411548120618528
mean function took 1134.057 ms
556.0
skewness function took 1134.569 ms
-0.2137018086949909
sum function took 1136.639 ms
10564
variance function took 1142.923 ms
78510.55555555555
drop_na function took 6.014 ms
cast function took 7.979 ms
median function took 78.789 ms
drop_na function took 4.024 ms
cast function took 6.020 ms
median function took 89.723 ms
mad function took 228.388 ms
230.0


In [9]:
df.cols.mad("price")

drop_na function took 5.025 ms
cast function took 9.012 ms
median function took 80.785 ms
drop_na function took 5.031 ms
cast function took 7.015 ms
median function took 96.740 ms
mad function took 224.401 ms


2.0

In [10]:
df.cols.mad("price", more= True)

drop_na function took 5.023 ms
cast function took 6.965 ms
median function took 81.780 ms
drop_na function took 2.992 ms
cast function took 7.014 ms
median function took 73.772 ms
mad function took 199.467 ms


{'mad': 2.0, 'median': 8.0}

In [11]:
print(df.cols.percentile(['price'], [0.05, 0.25, 0.5, 0.75, 0.95]))

drop_na function took 7.979 ms
cast function took 15.960 ms
percentile function took 93.750 ms
{0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}


In [12]:
print(df.cols.mode(["price","billingId"]))

mode function took 11191.787 ms
[{'price': 8}, {'billingId': None}]


### String Operations

In [20]:
df.table()

<class 'list'>


id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


table function took 1118.098 ms


In [18]:
df\
    .cols.trim("lastName")\
    .cols.lower("lastName")\
    .cols.upper(["product", "firstName"])\
    .cols.reverse("firstName")\
    .table()

Using 'Column Expression' to process column 'lastName'
trim function took 7.981 ms
Using 'Column Expression' to process column 'lastName'
lower function took 7.979 ms
Using 'Column Expression' to process column 'firstName'
Using 'Column Expression' to process column 'product'
upper function took 13.962 ms
Using 'Column Expression' to process column 'firstName'
reverse function took 6.981 ms
<class 'list'>


id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string)
1,SIUL,alvarez$$%!,123,CAKE,10,1980/07/07,never
2,ÉRDNA,ampère,423,PIZA,8,1950/07/08,gonna
3,SLEIN,böhr//((%%,551,PIZZA,8,1990/07/09,give
4,LUAP,dirac$,521,PIZZA,8,1954/07/10,you
5,TREBLA,einstein,634,PIZZA,8,1990/07/11,up
6,OELILAG,galilei,672,AREPA,5,1930/08/12,never
7,LRAC,ga%%%uss,323,TACO,3,1970/07/13,gonna
8,DIVAD,h$$$ilbert,624,TAAACCOO,3,1950/07/14,let
9,SENNAHOJ,kepler,735,TACO,3,1920/04/22,you
10,SEMAJ,m$$ax%%well,875,TACO,3,1923/03/12,down


table function took 1162.493 ms


In [15]:
df.cols.iqr("price")

drop_na function took 5.985 ms
cast function took 12.006 ms
percentile function took 85.792 ms
iqr function took 86.789 ms


5.0

In [16]:
df.cols.iqr("price", more= True)

drop_na function took 5.024 ms
cast function took 6.981 ms
percentile function took 85.771 ms
iqr function took 86.769 ms


{'iqr': 5.0, 'q1': 3.0, 'q3': 8.0}

In [19]:
df.cols.z_score("price").table()

mean function took 1109.065 ms
std function took 1103.709 ms
z_score function took 2266.630 ms
<class 'list'>


id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string),z_col_price (double)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never,1.3368014129178498
2,André,Ampère,423,piza,8,1950/07/08,gonna,0.6594886970394727
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give,0.6594886970394727
4,PAUL,dirac$,521,pizza,8,1954/07/10,you,0.6594886970394727
5,Albert,Einstein,634,pizza,8,1990/07/11,up,0.6594886970394727
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never,0.3564803767780932
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna,1.0337930926564705
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let,1.0337930926564705
9,Johannes,KEPLER,735,taco,3,1920/04/22,you,1.0337930926564705
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down,1.0337930926564705


table function took 1115.950 ms
